# Digit recognizer

In [3]:
import os
import pandas
import numpy as np

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import keras

from matplotlib import pyplot as plt
from PIL import Image

Using Theano backend.
Using gpu device 0: GeForce GTX 850M (CNMeM is disabled, cuDNN 5105)
/home/ursa/lib/anaconda2/envs/cv/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [4]:
TRAIN_FILENAME = 'data/train.csv'
TEST_FILENAME = 'data/test.csv'

# Read train csv

In [5]:
train = pandas.read_csv(TRAIN_FILENAME).values
test = pandas.read_csv(TEST_FILENAME).values

In [6]:
img_shape = (1, 28, 28)
X_train = train[:, 1:].reshape(train.shape[0], 1, 28, 28)
Y_train = keras.utils.np_utils.to_categorical(train[:, 0], 10)
X_test = test.reshape(test.shape[0], 1, 28, 28)

In [7]:
X_train = X_train.astype('float32')
X_train /= 255

X_test = X_test.astype('float32')
X_test /= 255

In [8]:
split_size = int(X_train.shape[0] * 0.9)
X_train, X_valid = X_train[:split_size], X_train[split_size:]
Y_train, Y_valid = Y_train[:split_size], Y_train[split_size:]

In [9]:
print('X_train shape: {}'.format(X_train.shape))
print('Y_train shape: {}\n'.format(Y_train.shape))

print('X_valid shape: {}'.format(X_valid.shape))
print('Y_valid shape: {}\n'.format(Y_valid.shape))

print('X_test shape: {}'.format(X_test.shape))

X_train shape: (37800, 1, 28, 28)
Y_train shape: (37800, 10)

X_valid shape: (4200, 1, 28, 28)
Y_valid shape: (4200, 10)

X_test shape: (28000, 1, 28, 28)


# Data augmentation

In [10]:
datagen_train = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15
)
datagen_valid = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15
)

In [11]:
datagen_train.fit(X_train)
datagen_valid.fit(X_valid)

# Model

In [12]:
model = Sequential()

model.add(BatchNormalization(axis=1, input_shape=img_shape))

model.add(Convolution2D(32,5,5, activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Convolution2D(64,5,5, activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Convolution2D(128,4,4, activation='relu'))

model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [13]:
n_epoch = 500
val_split = 0.1
batch_size = 16

In [ ]:
model.compile(
    "adamax",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit_generator(
    datagen_train.flow(X_train, Y_train, batch_size=batch_size),
    samples_per_epoch=len(X_train),
    nb_epoch=n_epoch,
    validation_data=datagen_valid.flow(X_valid, Y_valid, batch_size=batch_size),
    nb_val_samples=len(X_valid)
)

Epoch 1/500
37800/37800 [==============================] - 27s - loss: 1.0616 - acc: 0.6620 - val_loss: 0.3046 - val_acc: 0.9074
Epoch 2/500
34768/37800 [==========================>...] - ETA: 2s - loss: 0.3822 - acc: 0.8846

In [ ]:
model.summary()

# Predict test

In [ ]:
preds = model.predict_classes(X_test)

In [ ]:
np.savetxt(
    'pred.csv',
    np.c_[range(1, len(preds)+1), preds],
    delimiter=',',
    header='ImageId,Label',
    comments='',
    fmt='%d'
)

# Results

loss: 0.0086 - acc: 0.9975 - val_loss: 0.0509 - val_acc: 0.9914

http://pastebin.com/ec1Hn68B

loss: 0.0024 - acc: 0.9992 - val_loss: 0.0392 - val_acc: 0.9938
- Optimizer adam -> adamax
- Conv2D: number of filters x2

http://pastebin.com/p9AWwyUn

loss: 0.0307 - acc: 0.9919 - val_loss: 0.0345 - val_acc: 0.9924

- Data augmentation : rotation 20, w 0.15 h 0.15

http://pastebin.com/iY61hYHm

loss: 0.0350 - acc: 0.9913 - val_loss: 0.0374 - val_acc: 0.9900
- change Conv2D layers

http://pastebin.com/vPrhQmri